<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

<h1 align=center><font size = 5>Regression Models with Keras (Final Assignment)</font></h1>


## A. Build a baseline model

In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install scikit-learn


In [2]:
# get the data
import numpy as np
import pandas as pd

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
n_cols = len(predictors.columns)

### Model Creation

The function below creates a model that has one hidden layer with 10 hidden units.

In [3]:
# define regression model
from keras.models import Sequential
from keras.layers import Dense, Input

def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

2024-12-09 08:39:05.299439: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 08:39:05.358901: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Train and Test the Network


Let's call the function now to create our model.


In [4]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [5]:
from sklearn.model_selection import train_test_split
mse_test = []

# fit the model 50 times
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, epochs=50, verbose=0)
    mse_test.append(model.evaluate(X_test, y_test))
    print(f"{i+1}: mse_test = ", mse_test[-1])

mse_test_array = np.array(mse_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 191.6666  
1: mse_test =  197.8347625732422
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 125.0057
2: mse_test =  128.0734405517578
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1113.7996 
3: mse_test =  1173.899658203125
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 603.2647 
4: mse_test =  575.84814453125
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 732.9616 
5: mse_test =  709.9156494140625
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 291.5068 
6: mse_test =  278.9391174316406
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 222.5854 
7: mse_test =  209.6094207763672
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 641.0560 
8: mse_test =  596.667724609375
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 249.9497 
9: mse_test =  239.39065551757812
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 142.8741  
10: mse_test =  152.7887725830078
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 255.6875 
11: mse_test =  257.4233093261719
10/10 ━━

In [6]:
print("MSE mean = ", mse_test_array.mean())
print("MSE std = ", mse_test_array.std())

MSE mean =  278.5481727600098
MSE std =  233.47061573526523


## B. Normalize the data

Result: The standard deviation of the MSE decreases as compared to the baseline.

In [7]:
# normalize predictors
predictors_norm = (predictors-predictors.mean())/predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
mse_test = []

# fit the model 50 times, but now with normalized input data
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, epochs=50, verbose=0)
    mse_test.append(model.evaluate(X_test, y_test))
    print(f"{i+1}: mse_test = ", mse_test[-1])

mse_test_array = np.array(mse_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 336.2557 
1: mse_test =  341.2150573730469
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 355.9512  
2: mse_test =  345.99761962890625
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 372.2767 
3: mse_test =  364.65057373046875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 353.5955 
4: mse_test =  344.7601013183594
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 331.5974 
5: mse_test =  320.82220458984375
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 271.9980 
6: mse_test =  272.23822021484375
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 268.7806 
7: mse_test =  292.7491149902344
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 259.7632 
8: mse_test =  269.8823547363281
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 755.5303 
9: mse_test =  804.8427124023438
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 322.4538 
10: mse_test =  315.9245910644531
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 242.2485 
11: mse_test =  239.01516723632812
10

In [9]:
print("MSE mean = ", mse_test_array.mean())
print("MSE std = ", mse_test_array.std())

MSE mean =  349.5827606201172
MSE std =  108.56802532839362


## C. Increase number of epochs

MSE mean and std both decreases than in Part B. Meaning that increasing the number of epochs leads to better learning.

In [10]:
mse_test = []

# fit the model 50 times, but each training happens for 100 epochs
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, epochs=100, verbose=0)
    mse_test.append(model.evaluate(X_test, y_test))
    print(f"{i+1}: mse_test = ", mse_test[-1])

mse_test_array = np.array(mse_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 167.7165 
1: mse_test =  162.05657958984375
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 164.6206 
2: mse_test =  146.89682006835938
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 171.6410 
3: mse_test =  172.5339813232422
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 177.5807 
4: mse_test =  162.25347900390625
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 156.6875  
5: mse_test =  156.87673950195312
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 158.1525 
6: mse_test =  159.03240966796875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 166.5631 
7: mse_test =  164.42979431152344
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 219.2837 
8: mse_test =  205.56268310546875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 179.0157 
9: mse_test =  173.51707458496094
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 179.7832 
10: mse_test =  169.7512969970703
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 186.6885 
11: mse_test =  174.5341949462890

In [11]:
print("MSE mean = ", mse_test_array.mean())
print("MSE std = ", mse_test_array.std())

MSE mean =  164.9355612182617
MSE std =  17.110000717469823


## D. Increase number of hidden layers

The MSE mean is lower while std remains almost the same as Part C, while both of them are significantly lower than Part B. Increasing the depth of the model leads to improved learning.

In [12]:
# redefine the regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
model = regression_model()

In [13]:
mse_test = []

# fit the model 50 times
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, epochs=50, verbose=0)
    mse_test.append(model.evaluate(X_test, y_test))
    print(f"{i+1}: mse_test = ", mse_test[-1])

mse_test_array = np.array(mse_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 137.3918  
1: mse_test =  134.6854705810547
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 119.7487  
2: mse_test =  120.4246826171875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 129.2068 
3: mse_test =  133.9493408203125
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 155.8512  
4: mse_test =  142.2607879638672
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 111.2959
5: mse_test =  113.58807373046875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 136.7832  
6: mse_test =  130.51002502441406
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 97.5373 
7: mse_test =  103.96539306640625
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 119.7147 
8: mse_test =  120.69954681396484
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115.3576 
9: mse_test =  109.61353302001953
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 104.6222  
10: mse_test =  92.71543884277344
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 123.6217 
11: mse_test =  129.4695281982422


In [14]:
print("MSE mean = ", mse_test_array.mean())
print("MSE std = ", mse_test_array.std())

MSE mean =  125.92462982177734
MSE std =  18.235160465629935
